<h1>Beginning ROS2</h1>ROS2 is similar to MQTT in that you publish and subscribe to send messages over TCP/IP. However, in this case there is no broker as it only looks for other ROS nodes on the same LAN (local area network). You can go outside your network by saving a config file with the appropriate IP addresses.

## Publishing and Subscribing with CLI
You can publish anything you want by simply typing the following command line (CLI) into a terminal on a computer that is running ROS.  Note that the ! is a Jupyter Notebook trick for running a command line from within the notebook.  This line says that you are going to publish once a topic called "chatter" and send the string (a standard message type) "Hello".  For more on how to use the notebooks, go [here](../Help/200-JupyterLabs.ipynb).

In [ ]:
!ros2 topic pub -1 /chatter std_msgs/msg/String 'data: Hello'

Of course that does not do any good if no one is listening - so likewise you can subscribe to a topic in a commandline with the message below.  HOWEVER, if you do this from within the notebook, you will not be able to run any other commands - so you have a few choices.  (1) you can buddy up - and one of you publishes and one of you subscribes, or (2) you can open a terminal (blue plus sign and choose a terminal) and talk from notebook to terminal, or (3) you can start a new notebook.

In [ ]:
!ros2 topic echo '/chatter'

There are a lot of commandline codes (look [here](https://www.theconstructsim.com/wp-content/uploads/2021/10/ROS2-Command-Cheat-Sheets-updated.pdf) for a complete list).  Some of the more useful ones are listing notes and topics.

In [ ]:
!ros2 topic list

In [ ]:
!ros2 node list

So what is the difference?  Each node contains a number of topics.  Each topic can be published to or subscribed to.  Typically, your robot is a node (like /myRobot) with a number of topics (like /myRobot/move).  You can always find out more about the topic with this command:

In [ ]:
!ros2 topic info /chatter

## Publish and Subscribe with Python
Clearly the CLI will not work in the long term in that you probably want to integrate a bunch of other sutff with your robot control code - so that is where Python comes in.

All of your python code is asynchronous - meaning that you have to set up mulitple tasks that run in parallel.  Like asyncio, you will define a few functions with callbacks and then spin it all up.  Here is a simple code that publishes every second to the topic "tell" and subscribes to the topic "listen".

In [ ]:
from rclpy.node import Node
from std_msgs.msg import String

import time

class ROS_Minimal(Node):

    def __init__(self, port = None):  
        # initialize the topic (name it, create the publisher and publishing rate
        super().__init__('MinimalNode')
        queue_size = 10
        self.serialPub = self.create_publisher(String, '/tell', queue_size)  
        self.serialSub = self.create_subscription(String,'/listen',self.listener_callback,10)
        self.serialSub  # prevent unused variable warning

        timer_period = 1  # seconds
        self.timer = self.create_timer(timer_period, self.timer_callback)
        print('initialized node ')
        
#-------------------------define callbacks

    def timer_callback(self):
        # every interval, check for new data
        msg = String()
        msg.data = str(time.time())
        self.serialPub.publish(msg)
        self.get_logger().info('published: "%s"' % msg.data)  # prints to console / log

    def listener_callback(self, msg):
        payload = msg.data
        print("I heard "+payload)
        self.get_logger().info('I heard: "%s"' % payload)        
        
#------------------------run everything
import rclpy

def main():
    print('starting up')
    rclpy.init()
    node = ROS_Minimal()
    try:
        print('spinning up node')
        rclpy.spin(node)
        # stay here forever, reading and publishing messages
    except Exception as e:
        print(e)
    finally:
        node.destroy_node()
        rclpy.shutdown()
        print('Done')
        
main()

Try going to the terminal window and to a ros2 topic list - do you see your topic?  Get someone else to publish to '/listen' - does your subscription work?

## Services and actions
But wait - there is more.  A node can also have services or actions.  Services are things that you ask for once and get back an immediate answer (for instance a version number- something that does not change).  A service has two parts - a *Request* and a *Response*. An action is the same thing only it is for things that take a little longer.  So it has three states (and two callbacks): *Resquest*, *Result*, and *Feedback*.  The request is what you send to the robot - and when it is done you get back a result - however you can, at any time, ask the robot how close it is to the goal with the feedback parameter.

## Parameters
You can also set (or list) parameters.  These are things like dimensions of your robot etc - things that do not change a lot.

In [1]:
!ros2 param list